In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!pip install aif360
!pip install BlackBoxAuditing
!pip install fairlearn==0.4.6

# Load and Split Data

In [3]:
from aif360.datasets import AdultDataset, BinaryLabelDataset

/home/sophie/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sophie/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sophie/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sophie/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [120]:
adult_numerical = pd.read_csv('fairness/data/preprocessed/adult_numerical.csv')
filtered_idx = adult_numerical['race'] != 'Other'
adult_numerical = adult_numerical[filtered_idx]
adult_numerical['race-sex'] = adult_numerical['race-sex'].astype('category')
map_race_sex = dict( enumerate(adult_numerical['race-sex'].cat.categories))
adult_numerical['race-sex'] = adult_numerical['race-sex'].cat.codes

adult_numerical['race'] = adult_numerical['race'].astype('category')
map_race = dict( enumerate(adult_numerical['race'].cat.categories))
adult_numerical['race'] = adult_numerical['race'].cat.codes


adult_numerical['sex'] = adult_numerical['sex'].astype('category')
map_sex = dict( enumerate(adult_numerical['sex'].cat.categories))
adult_numerical['sex'] = adult_numerical['sex'].cat.codes

adult_numerical['income-per-year'] = adult_numerical['income-per-year'].astype('category')
map_income = dict( enumerate(adult_numerical['income-per-year'].cat.categories))
adult_numerical['income-per-year'] = adult_numerical['income-per-year'].cat.codes


In [37]:
# TO FINISH
adult_binary = BinaryLabelDataset(df=adult_numerical,
                                  label_names=['income-per-year'], 
                                  protected_attribute_names=['race-sex'], 
                                  instance_weights_name=None, 
                                  scores_names=[], 
                                  unprivileged_protected_attributes=[], 
                                  privileged_protected_attributes=[], 
                                  metadata=None)

In [38]:
ad_train, ad_test = adult_binary.split(2)

In [40]:
print(adult_binary.privileged_protected_attributes)

[array([7.])]


# Unconstrained Model

In [41]:
from sklearn.linear_model import LogisticRegression
from copy import deepcopy
from aif360.metrics import ClassificationMetric

In [42]:
ad_df_train, ad_attrs_train = ad_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
ad_df_test, ad_attrs_test = ad_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)

In [43]:
ad_df_train.head()

,age,education-num,race,sex,capital-gain,capital-loss,hours-per-week,race-sex,workclass_Federal-gov,workclass_Local-gov,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income-per-year
0,39.0,13.0,3.0,1.0,2174.0,0.0,40.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,50.0,13.0,3.0,1.0,0.0,0.0,13.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,38.0,9.0,3.0,1.0,0.0,0.0,40.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,53.0,7.0,2.0,1.0,0.0,0.0,40.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,28.0,13.0,2.0,0.0,0.0,0.0,40.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
Y_train = np.array(ad_df_train[['income-per-year']]).reshape(((len(ad_df_train),))).astype(int)
Xs_train = np.array(ad_df_train.drop(columns='income-per-year'))
Y_test = np.array(ad_df_test[['income-per-year']]).reshape(((len(ad_df_test),))).astype(int)
Xs_test = np.array(ad_df_test.drop(columns='income-per-year'))

In [84]:
clf = LogisticRegression(max_iter = 300, solver='liblinear').fit(Xs_train, Y_train)

In [85]:
predicted_labels_train = clf.predict(Xs_train).reshape((len(Y_train), 1))
predicted_labels = clf.predict(Xs_test).reshape((len(Y_test), 1))

In [86]:
ad_pred = deepcopy(ad_test)
ad_pred_train = deepcopy(ad_train)
ad_pred.labels = predicted_labels
ad_pred_train.labels = predicted_labels_train

In [87]:
u = [{'race-sex': 0, 'race-sex': 1, 'race-sex': 2, 'race-sex': 3, 'race-sex': 4, 'race-sex': 5, 'race-sex': 6}]
p = [{'race-sex': 7}]
metrics = ClassificationMetric(ad_test,ad_pred,unprivileged_groups=u, privileged_groups=p)
test_acc = metrics.accuracy()
DI = metrics.disparate_impact()
EO = metrics.average_abs_odds_difference()
print('Original Test Accuracy:', test_acc)
print('Original Demographic Parity ratio:', DI)
print('Original Average Absolute Odds diff:', EO)

Original Test Accuracy: 0.8460407617774808
Original Demographic Parity ratio: 0.2996890474115724
Original Average Absolute Odds diff: 0.0921651727995266


In [88]:
from mia.membership_inference_attacks import black_box_benchmarks

In [103]:
# Membership inference attack

# train shadow model
shadow_train, shadow_test = ad_test.split(2)
shadow_train_df, _ = shadow_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
shadow_test_df, _ = shadow_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
shadow_Y_train = np.array(shadow_train_df[['income-per-year']]).reshape(((len(shadow_train_df),))).astype(int)
shadow_Xs_train = np.array(shadow_train_df.drop(columns='income-per-year'))
shadow_Y_test = np.array(shadow_test_df[['income-per-year']]).reshape(((len(shadow_test_df),))).astype(int)
shadow_Xs_test = np.array(shadow_test_df.drop(columns='income-per-year'))

true_pred_on_shadow_train = clf.predict(shadow_Xs_train)
true_pred_on_shadow_test = clf.predict(shadow_Xs_test)

shadow_model = LogisticRegression(max_iter = 300, solver='liblinear').fit(shadow_Xs_train, true_pred_on_shadow_train)


In [104]:
# get outputs
shadow_out_train = shadow_model.predict_proba(shadow_Xs_train)
shadow_out_test = shadow_model.predict_proba(shadow_Xs_test)
true_model_train = clf.predict_proba(Xs_train)
true_model_test = clf.predict_proba(Xs_test)

In [105]:
shadow_train_performance = (shadow_out_train, true_pred_on_shadow_train)
shadow_test_performance = (shadow_out_test, true_pred_on_shadow_test)
target_train_performance = (true_model_train, Y_train)
target_test_performance = (true_model_test, Y_test)

In [106]:
shadow_train_df.head()

,age,education-num,race,sex,capital-gain,capital-loss,hours-per-week,race-sex,workclass_Federal-gov,workclass_Local-gov,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income-per-year
15072,34.0,10.0,2.0,1.0,0.0,0.0,40.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
15073,27.0,13.0,3.0,0.0,0.0,0.0,40.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
15074,26.0,11.0,3.0,1.0,0.0,0.0,48.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
15075,27.0,9.0,3.0,1.0,0.0,0.0,40.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
15076,55.0,11.0,3.0,1.0,0.0,0.0,40.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [107]:
shadow_train_df['race-sex'] = shadow_train_df['race'].astype(str) + '-' + shadow_train_df['sex'].astype(str)
shadow_train_df['race-sex'] = shadow_train_df['race-sex'].astype('category').cat.codes
shadow_test_df['race-sex'] = shadow_test_df['race'].astype(str) + '-' + shadow_test_df['sex'].astype(str)
shadow_test_df['race-sex'] = shadow_test_df['race-sex'].astype('category').cat.codes
ad_df_train['race-sex'] = ad_df_train['race'].astype(str) + '-' + ad_df_train['sex'].astype(str)
ad_df_train['race-sex'] = ad_df_train['race-sex'].astype('category').cat.codes
ad_df_test['race-sex'] = ad_df_test['race'].astype(str) + '-' + ad_df_test['sex'].astype(str)
ad_df_test['race-sex'] = ad_df_test['race-sex'].astype('category').cat.codes

In [108]:
# run MIA
MIA = black_box_benchmarks(shadow_train_performance,shadow_test_performance,
                         target_train_performance,target_test_performance,num_classes=2)

In [109]:
MIA._mem_inf_benchmarks(shadow_train_df['race-sex'].astype(int), shadow_test_df['race-sex'].astype(int), ad_df_train['race-sex'].astype(int), ad_df_test['race-sex'].astype(int))

For membership inference attack via correctness, the attack acc is 0.499, with train acc 0.845 and test acc 0.846
For membership inference attack via confidence, group attr 0, the attack acc is 0.551
For membership inference attack via confidence, group attr 1, the attack acc is 0.586
For membership inference attack via confidence, group attr 2, the attack acc is 0.561
For membership inference attack via confidence, group attr 3, the attack acc is 0.556
For membership inference attack via confidence, group attr 4, the attack acc is 0.544
For membership inference attack via confidence, group attr 5, the attack acc is 0.518
For membership inference attack via confidence, group attr 6, the attack acc is 0.515
For membership inference attack via confidence, group attr 7, the attack acc is 0.511
For membership inference attack via confidence, the shadow attack acc is 0.517
For membership inference attack via confidence, the attack acc is 0.498
For membership inference attack via entropy, gr

# Privacy Risk Score Analysis

In [110]:
from mia.privacy_risk_score_utils import calculate_risk_score

In [111]:
tr_values = MIA._entr_comp(shadow_out_train)
te_values = MIA._entr_comp(shadow_out_test)
data_values = MIA._entr_comp(true_model_train)

In [112]:
# get risk scores
risk_scores = calculate_risk_score(tr_values, te_values, 
                                   true_pred_on_shadow_train, 
                                   true_pred_on_shadow_test, 
                                   data_values, Y_train, num_bins=5, log_bins=True)

In [121]:
print(map_race_sex)
print(map_race)
print(map_sex)
print(map_income)

{0: 'Amer-Indian-Eskimo-Female', 1: 'Amer-Indian-Eskimo-Male', 2: 'Asian-Pac-Islander-Female', 3: 'Asian-Pac-Islander-Male', 4: 'Black-Female', 5: 'Black-Male', 6: 'White-Female', 7: 'White-Male'}
{0: 'Amer-Indian-Eskimo', 1: 'Asian-Pac-Islander', 2: 'Black', 3: 'White'}
{0: 'Female', 1: 'Male'}
{0: '<=50K', 1: '>50K'}


In [123]:
white_idx = ad_df_train['race'] == 3
aie_idx = ad_df_train['race'] == 0
api_idx = ad_df_train['race'] == 1
black_idx = ad_df_train['race'] == 2
male_idx = ad_df_train['sex'] == 1
female_idx = ad_df_train['sex'] == 0
income_above_idx = ad_df_train['income-per-year'] == 1
income_below_idx = ad_df_train['income-per-year'] == 0

In [127]:
plt.clf()
plt.hist(risk_scores[white_idx], label='white', density=True)
plt.hist(risk_scores[black_idx], label='black', density=True)
plt.hist(risk_scores[api_idx], label = 'asian-pac-islander', density=True)
plt.hist(risk_scores[aie_idx], label = 'amer-indian-eskimo', density=True)
plt.legend()
plt.savefig('race.png')

In [128]:
plt.clf()
plt.hist(risk_scores[male_idx], label='male', density=True)
plt.hist(risk_scores[female_idx], label='female', density=True)
plt.legend()
plt.savefig('gender.png')

In [129]:
plt.clf()
plt.hist(risk_scores[income_above_idx], label='>50k', density=True)
plt.hist(risk_scores[income_below_idx], label='<=50k', density=True)
plt.legend()
plt.savefig('income.png')

# Demographic Parity (Independence)

## Feldman et al. Repair (Preprocessing)

In [29]:
import aif360.algorithms.preprocessing as AIF

In [70]:
# tuning repair level to minimize test accuracy tradeoff
x = np.arange(0, 1.1, 0.1)
test_acc = []
dp = []
for rep in x:
    repairer = AIF.DisparateImpactRemover(repair_level=rep, sensitive_attribute='race-sex')
    repaired_train = repairer.fit_transform(ad_train)
    repaired_test = repairer.fit_transform(ad_test)
    repaired_df_train, repaired_attrs_train = repaired_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
    repaired_df_test, repaired_attrs_test = repaired_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
    Y_rep_train = np.array(repaired_df_train[['income-per-year']]).reshape(((len(repaired_df_train),)))
    Xs_rep_train = np.array(repaired_df_train.drop(columns='income-per-year'))

    Y_rep_test = np.array(repaired_df_test[['income-per-year']]).reshape(((len(repaired_df_test),)))
    Xs_rep_test = np.array(repaired_df_test.drop(columns='income-per-year'))
    clf_rep = LogisticRegression(max_iter = 300, solver='liblinear').fit(Xs_rep_train, Y_rep_train)
    predicted_labels_rep = clf_rep.predict(Xs_rep_test).reshape((len(Y_rep_test), 1))
    ad_pred_rep = deepcopy(ad_test)
    ad_pred_rep.labels = predicted_labels_rep
    metrics_rep = ClassificationMetric(ad_test,ad_pred_rep,unprivileged_groups=u, privileged_groups=p)
    test_acc_rep = metrics_rep.accuracy()
    DI_rep = metrics_rep.disparate_impact()
    EO_rep = metrics_rep.average_abs_odds_difference()
    print('Repair Test Accuracy:', test_acc_rep)
    print('Repair Demographic Parity ratio:', DI_rep)
    print('Repair Average Absolute Odds diff:', EO_rep)
    test_acc.append(test_acc_rep)
    dp.append(DI_rep)

Repair Test Accuracy: 0.8460407617774808
Repair Demographic Parity ratio: 0.2996890474115724
Repair Average Absolute Odds diff: 0.0921651727995266
Repair Test Accuracy: 0.8408954226528567
Repair Demographic Parity ratio: 0.3394946668003241
Repair Average Absolute Odds diff: 0.04862292852946082
Repair Test Accuracy: 0.8342799866354828
Repair Demographic Parity ratio: 0.3468472819773562
Repair Average Absolute Odds diff: 0.03150792385649004
Repair Test Accuracy: 0.8203140661543602
Repair Demographic Parity ratio: 0.3687730033392927
Repair Average Absolute Odds diff: 0.017384281993712208
Repair Test Accuracy: 0.843701971266288
Repair Demographic Parity ratio: 0.3553400589107184
Repair Average Absolute Odds diff: 0.04394799807579919
Repair Test Accuracy: 0.6314734380220515
Repair Demographic Parity ratio: 0.4870669034922455
Repair Average Absolute Odds diff: 0.19011057600434675
Repair Test Accuracy: 0.63701971266288
Repair Demographic Parity ratio: 0.5401011251489555
Repair Average Absolut

In [72]:
plt.clf()
plt.plot(x, test_acc, label='test acc')
plt.plot(x, dp, label='demographic parity ratio')
plt.legend()
plt.savefig("feldman_repair.png")

In [73]:
# repair level 0.9 seems to have ok demographic parity ratio and with a small drop in test accuracy
repairer = AIF.DisparateImpactRemover(repair_level=0.9, sensitive_attribute='race-sex')
repaired_train = repairer.fit_transform(ad_train)
repaired_test = repairer.fit_transform(ad_test)
repaired_df_train, repaired_attrs_train = repaired_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
repaired_df_test, repaired_attrs_test = repaired_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
Y_rep_train = np.array(repaired_df_train[['income-per-year']]).reshape(((len(repaired_df_train),)))
Xs_rep_train = np.array(repaired_df_train.drop(columns='income-per-year'))

Y_rep_test = np.array(repaired_df_test[['income-per-year']]).reshape(((len(repaired_df_test),)))
Xs_rep_test = np.array(repaired_df_test.drop(columns='income-per-year'))
clf_rep = LogisticRegression(max_iter = 300, solver='liblinear').fit(Xs_rep_train, Y_rep_train)
predicted_labels_rep = clf_rep.predict(Xs_rep_test).reshape((len(Y_rep_test), 1))
ad_pred_rep = deepcopy(ad_test)
ad_pred_rep.labels = predicted_labels_rep
metrics_rep = ClassificationMetric(ad_test,ad_pred_rep,unprivileged_groups=u, privileged_groups=p)
test_acc_rep = metrics_rep.accuracy()
DI_rep = metrics_rep.disparate_impact()
EO_rep = metrics_rep.average_abs_odds_difference()
print('Repair Test Accuracy:', test_acc_rep)
print('Repair Demographic Parity ratio:', DI_rep)
print('Repair Average Absolute Odds diff:', EO_rep)

Repair Test Accuracy: 0.6248580020046776
Repair Demographic Parity ratio: 0.5953345912631965
Repair Average Absolute Odds diff: 0.13032918174151628


In [152]:
# Membership inference attack

repaired_shadow_train = repairer.fit_transform(shadow_train)
repaired_shadow_test = repairer.fit_transform(shadow_test)

shadow_train_df, _ = repaired_shadow_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
shadow_test_df, _ = repaired_shadow_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
shadow_Xs_train = np.array(shadow_train_df.drop(columns='income-per-year'))
shadow_Xs_test = np.array(shadow_test_df.drop(columns='income-per-year'))


# train shadow model
true_pred_on_shadow_train = clf_rep.predict(shadow_Xs_train)
true_pred_on_shadow_test = clf_rep.predict(shadow_Xs_test)

shadow_model = LogisticRegression(max_iter = 300, solver='liblinear').fit(shadow_Xs_train, true_pred_on_shadow_train)

In [153]:
true_pred_on_shadow_train = true_pred_on_shadow_train.astype(int).reshape(len(true_pred_on_shadow_train),)
true_pred_on_shadow_test = true_pred_on_shadow_test.astype(int).reshape(len(true_pred_on_shadow_test),)

In [154]:
X_df_train = pd.DataFrame(ad_train.features, columns=ad_train.feature_names)
X_df_test = pd.DataFrame(ad_test.features, columns=ad_test.feature_names)

In [155]:
shadow_out_train = clf_rep.predict_proba(shadow_Xs_train)
shadow_out_test = clf_rep.predict_proba(shadow_Xs_test)

In [156]:
shadow_train_performance = (shadow_out_train, true_pred_on_shadow_train)
shadow_test_performance = (shadow_out_test, true_pred_on_shadow_test)
target_train_performance = (true_model_train, Y_train)
target_test_performance = (true_model_test, Y_test)

In [157]:
# run MIA
MIA = black_box_benchmarks(shadow_train_performance,shadow_test_performance,
                         target_train_performance,target_test_performance,num_classes=2)

In [158]:
MIA._mem_inf_benchmarks(shadow_train_df['race-sex'].astype(int), shadow_test_df['race-sex'].astype(int), ad_df_train['race-sex'].astype(int), ad_df_test['race-sex'].astype(int))

For membership inference attack via correctness, the attack acc is 0.499, with train acc 0.845 and test acc 0.846
For membership inference attack via confidence, group attr 0, the attack acc is 0.764
For membership inference attack via confidence, group attr 1, the attack acc is 0.614
For membership inference attack via confidence, group attr 2, the attack acc is 0.706
For membership inference attack via confidence, group attr 3, the attack acc is 0.611
For membership inference attack via confidence, group attr 4, the attack acc is 0.717
For membership inference attack via confidence, group attr 5, the attack acc is 0.600
For membership inference attack via confidence, group attr 6, the attack acc is 0.671
For membership inference attack via confidence, group attr 7, the attack acc is 0.653
For membership inference attack via confidence, the shadow attack acc is 0.659
For membership inference attack via confidence, the attack acc is 0.501
For membership inference attack via entropy, gr

## Agarwal et al (Reduction)

In [56]:
from aif360.algorithms.inprocessing.exponentiated_gradient_reduction import ExponentiatedGradientReduction

In [57]:
estimator = LogisticRegression(solver='liblinear')
np.random.seed(0) #need for reproducibility
exp_grad_red_dp = ExponentiatedGradientReduction(estimator=estimator, 
                                              constraints="DemographicParity",
                                              drop_prot_attr=False)

In [58]:
exp_grad_red_dp.fit(ad_train)
exp_grad_red_pred_dp = exp_grad_red_dp.predict(ad_test)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [59]:
metrics_red_dp = ClassificationMetric(ad_test,exp_grad_red_pred_dp,unprivileged_groups=u, privileged_groups=p)
test_acc_red_dp = metrics_red_dp.accuracy()
DI_red_dp = metrics_red_dp.disparate_impact()
EO_red_dp = metrics_red_dp.average_abs_odds_difference()
print('DP Reduction Test Accuracy:', test_acc_red_dp)
print('DP Reduction Demographic Parity ratio:', DI_red_dp)
print('DP Reduction Average Absolute Odds diff:', EO_red_dp)

DP Reduction Test Accuracy: 0.8026729034413632
DP Reduction Demographic Parity ratio: 0.8863646735827712
DP Reduction Average Absolute Odds diff: 0.13793206099376182


In [159]:
# Membership inference attack

# train shadow model
true_pred_on_shadow_train = exp_grad_red_dp.predict(shadow_train)
true_pred_on_shadow_test = exp_grad_red_dp.predict(shadow_test)

shadow_model = LogisticRegression(max_iter = 300, solver='liblinear').fit(shadow_Xs_train, true_pred_on_shadow_train.labels)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [160]:
true_pred_on_shadow_train = true_pred_on_shadow_train.labels.astype(int).reshape(len(true_pred_on_shadow_train.labels),)
true_pred_on_shadow_test = true_pred_on_shadow_test.labels.astype(int).reshape(len(true_pred_on_shadow_test.labels),)

In [161]:
X_df_train = pd.DataFrame(ad_train.features, columns=ad_train.feature_names)
X_df_test = pd.DataFrame(ad_test.features, columns=ad_test.feature_names)

In [162]:
shadow_out_train = exp_grad_red_dp.model.predict_proba(shadow_Xs_train)
shadow_out_test = exp_grad_red_dp.model.predict_proba(shadow_Xs_test)

In [163]:
shadow_train_performance = (shadow_out_train, true_pred_on_shadow_train)
shadow_test_performance = (shadow_out_test, true_pred_on_shadow_test)
target_train_performance = (true_model_train, Y_train)
target_test_performance = (true_model_test, Y_test)

In [164]:
# run MIA
MIA = black_box_benchmarks(shadow_train_performance,shadow_test_performance,
                         target_train_performance,target_test_performance,num_classes=2)

In [165]:
MIA._mem_inf_benchmarks(shadow_train_df['race-sex'].astype(int), shadow_test_df['race-sex'].astype(int), ad_df_train['race-sex'].astype(int), ad_df_test['race-sex'].astype(int))

For membership inference attack via correctness, the attack acc is 0.499, with train acc 0.845 and test acc 0.846
For membership inference attack via confidence, group attr 0, the attack acc is 0.634
For membership inference attack via confidence, group attr 1, the attack acc is 0.555
For membership inference attack via confidence, group attr 2, the attack acc is 0.700
For membership inference attack via confidence, group attr 3, the attack acc is 0.574
For membership inference attack via confidence, group attr 4, the attack acc is 0.634
For membership inference attack via confidence, group attr 5, the attack acc is 0.551
For membership inference attack via confidence, group attr 6, the attack acc is 0.599
For membership inference attack via confidence, group attr 7, the attack acc is 0.561
For membership inference attack via confidence, the shadow attack acc is 0.577
For membership inference attack via confidence, the attack acc is 0.500
For membership inference attack via entropy, gr

# Equalized Odds (Separation)

## Hardt et al. (Postprocessing)

In [74]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing

In [75]:
postprocessing = EqOddsPostprocessing(u, p)

In [76]:
postprocessing = postprocessing.fit(ad_train, ad_pred_train)
postprocess_pred = postprocessing.predict(ad_pred)

In [77]:
metrics_post = ClassificationMetric(ad_test,postprocess_pred,unprivileged_groups=u, privileged_groups=p)
test_acc_post = metrics_post.accuracy()
DI_post = metrics_post.disparate_impact()
EO_ppost = metrics_post.average_abs_odds_difference()
print('Postprocessing Test Accuracy:', test_acc_post)
print('Postprocessing Demographic Parity ratio:', DI_post)
print('Postprocessing Average Absolute Odds diff:', EO_ppost)

Postprocessing Test Accuracy: 0.8138322753090544
Postprocessing Demographic Parity ratio: 0.6174765669542002
Postprocessing Average Absolute Odds diff: 0.004022424837044972


In [133]:
# Membership inference attack

# train shadow model
true_pred_on_shadow_train = postprocessing.predict(shadow_train)
true_pred_on_shadow_test = postprocessing.predict(shadow_test)

shadow_model = LogisticRegression(max_iter = 300, solver='liblinear').fit(shadow_Xs_train, true_pred_on_shadow_train.labels)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [134]:
true_pred_on_shadow_train = true_pred_on_shadow_train.labels.astype(int).reshape(len(true_pred_on_shadow_train.labels),)
true_pred_on_shadow_test = true_pred_on_shadow_test.labels.astype(int).reshape(len(true_pred_on_shadow_test.labels),)

In [135]:
X_df_train = pd.DataFrame(ad_train.features, columns=ad_train.feature_names)
X_df_test = pd.DataFrame(ad_test.features, columns=ad_test.feature_names)

In [138]:
# TODO: probabilities from hardt postprocessing technique
shadow_out_train = clf.predict_proba(shadow_Xs_train)
shadow_out_test = clf.predict_proba(shadow_Xs_test)

In [139]:
shadow_train_performance = (shadow_out_train, true_pred_on_shadow_train)
shadow_test_performance = (shadow_out_test, true_pred_on_shadow_test)
target_train_performance = (true_model_train, Y_train)
target_test_performance = (true_model_test, Y_test)

In [140]:
# run MIA
MIA = black_box_benchmarks(shadow_train_performance,shadow_test_performance,
                         target_train_performance,target_test_performance,num_classes=2)

In [141]:
MIA._mem_inf_benchmarks(shadow_train_df['race-sex'].astype(int), shadow_test_df['race-sex'].astype(int), ad_df_train['race-sex'].astype(int), ad_df_test['race-sex'].astype(int))

For membership inference attack via correctness, the attack acc is 0.499, with train acc 0.845 and test acc 0.846
For membership inference attack via confidence, group attr 0, the attack acc is 0.549
For membership inference attack via confidence, group attr 1, the attack acc is 0.592
For membership inference attack via confidence, group attr 2, the attack acc is 0.550
For membership inference attack via confidence, group attr 3, the attack acc is 0.519
For membership inference attack via confidence, group attr 4, the attack acc is 0.538
For membership inference attack via confidence, group attr 5, the attack acc is 0.508
For membership inference attack via confidence, group attr 6, the attack acc is 0.513
For membership inference attack via confidence, group attr 7, the attack acc is 0.510
For membership inference attack via confidence, the shadow attack acc is 0.513
For membership inference attack via confidence, the attack acc is 0.496
For membership inference attack via entropy, gr

## Agarwal et al (Reduction)

In [78]:
estimator = LogisticRegression(solver='liblinear')
np.random.seed(0) #need for reproducibility
exp_grad_red = ExponentiatedGradientReduction(estimator=estimator, 
                                              constraints="EqualizedOdds",
                                              drop_prot_attr=False)

In [79]:
exp_grad_red.fit(ad_train)
exp_grad_red_pred = exp_grad_red.predict(ad_test)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [80]:
metrics_red_eo = ClassificationMetric(ad_test,exp_grad_red_pred,unprivileged_groups=u, privileged_groups=p)
test_acc_red_eo = metrics_red_eo.accuracy()
DI_red_eo = metrics_red_eo.disparate_impact()
EO_red_eo = metrics_red_eo.average_abs_odds_difference()
print('EO Reduction Test Accuracy:', test_acc_red_eo)
print('EO Reduction Demographic Parity ratio:', DI_red_eo)
print('EO Reduction Average Absolute Odds diff:', EO_red_eo)

EO Reduction Test Accuracy: 0.7929836284664217
EO Reduction Demographic Parity ratio: 0.573711529289617
EO Reduction Average Absolute Odds diff: 0.022731095923982356


In [96]:
# Membership inference attack

# train shadow model
true_pred_on_shadow_train = exp_grad_red.predict(shadow_train)
true_pred_on_shadow_test = exp_grad_red.predict(shadow_test)

shadow_model = LogisticRegression(max_iter = 300, solver='liblinear').fit(shadow_Xs_train, true_pred_on_shadow_train.labels)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [99]:
true_pred_on_shadow_train = true_pred_on_shadow_train.labels.astype(int).reshape(len(true_pred_on_shadow_train.labels),)
true_pred_on_shadow_test = true_pred_on_shadow_test.labels.astype(int).reshape(len(true_pred_on_shadow_test.labels),)

In [97]:
X_df_train = pd.DataFrame(ad_train.features, columns=ad_train.feature_names)
X_df_test = pd.DataFrame(ad_test.features, columns=ad_test.feature_names)

In [137]:
shadow_out_train = exp_grad_red.model.predict_proba(shadow_Xs_train)
shadow_out_test = exp_grad_red.model.predict_proba(shadow_Xs_test)

In [100]:
shadow_train_performance = (shadow_out_train, true_pred_on_shadow_train)
shadow_test_performance = (shadow_out_test, true_pred_on_shadow_test)
target_train_performance = (true_model_train, Y_train)
target_test_performance = (true_model_test, Y_test)

In [101]:
# run MIA
MIA = black_box_benchmarks(shadow_train_performance,shadow_test_performance,
                         target_train_performance,target_test_performance,num_classes=2)

In [102]:
MIA._mem_inf_benchmarks(shadow_train_df['race-sex'].astype(int), shadow_test_df['race-sex'].astype(int), ad_df_train['race-sex'].astype(int), ad_df_test['race-sex'].astype(int))

For membership inference attack via correctness, the attack acc is 0.501, with train acc 0.834 and test acc 0.832
For membership inference attack via confidence, group attr 0, the attack acc is 0.617
For membership inference attack via confidence, group attr 1, the attack acc is 0.580
For membership inference attack via confidence, group attr 2, the attack acc is 0.570
For membership inference attack via confidence, group attr 3, the attack acc is 0.565
For membership inference attack via confidence, group attr 4, the attack acc is 0.553
For membership inference attack via confidence, group attr 5, the attack acc is 0.528
For membership inference attack via confidence, group attr 6, the attack acc is 0.508
For membership inference attack via confidence, group attr 7, the attack acc is 0.510
For membership inference attack via confidence, the shadow attack acc is 0.515
For membership inference attack via confidence, the attack acc is 0.502
For membership inference attack via entropy, gr